# P4Lang Tutorials of FABRIC

This notebook walks the user through setting up a FABRIC eperiment that is suitiable for completing the P4 tutorials created by [P4Lang](https://github.com/p4lang/tutorials). The tutorials were origianlly designed to use a mininet topology. This example replaces the mininet topology with a FABRIC experiemnt topology that may span multiple sites across the FABRIC testbed.

Additional resources:
- [FABRIC Knowledge Base](https://learn.fabric-testbed.net/)
- [FABRIC Forums](https://learn.fabric-testbed.net/forums/)
- [P4Lang Tutorials](https://github.com/p4lang/tutorials)
- [P4Lang YouTube Presentations](https://www.youtube.com/channel/UCOQAFkDKucJWr-KafdJsdIQ)

## Basic FABRIC Slice Configuration

In [1]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

In [2]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Name      CPUs  Cores    RAM (G)    Disk (G)       Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -------------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
MASS         4  125/128  1016/1024  55750/55800    253/254                 2/2                             0/0                            6/6                 0/0               3/3
MAX         10  194/320  2110/2560  114902/116400  564/635                 2/2                             1/4                            16/16               4/4               4/6
TACC        10  288/320  2420/2560  116176/116400  618/635                 0/2                             1/4                            15/16               4/4               6/6
MICH         6  188/192  1520/1536  60500/60600    381/381                 0/2

In [5]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Name      CPUs  Cores    RAM (G)    Disk (G)       Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -------------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
MASS         4  125/128  1016/1024  55750/55800    253/254                 2/2                             0/0                            6/6                 0/0               3/3
MAX         10  194/320  2110/2560  114902/116400  564/635                 2/2                             1/4                            16/16               4/4               4/6
TACC        10  288/320  2420/2560  116176/116400  618/635                 0/2                             1/4                            15/16               4/4               6/6
MICH         6  188/192  1520/1536  60500/60600    381/381                 0/2

## Configure Slice Parameters

This section builds the experiment slice 

<img src="figs/fabric_slice.png" width="800"/>



In [3]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

# Slice 
slice_name = 'VCC3_2'

# Switches
s1_name = "s1"
s2_name = "s2"
s3_name = "s3"

switch_cores = 2
switch_ram = 8
switch_disk = 40

# Hosts
h1_name = "h1"
h2_name = "h2"

h1_ip="10.0.1.1"
h2_ip="10.0.1.2"

host_cores = 2
host_ram = 8
host_disk = 10
# Sites
site_1 = 'MAX' # 'SALT' # 'NCSA' # s'WASH' # 'MAX' # 'STAR' # 'NCSA' # 'MAX' # 'UTAH' # 'DALL' # 'TACC' # 'MAX'
site_2 = 'TACC' # 'SALT' # 'NCSA' # s'WASH' # 'MAX' # 'STAR' # 'NCSA' # 'MAX' # 'UTAH' # 'DALL' # 'TACC' # 'MAX'

net_h1_name = 'net_h1'
net_h2_name = 'net_h2'


# All node properties
#username = 'ubuntu'
image = 'default_ubuntu_20'
vlan = '1000'
#image_type = 'qcow2'



In [8]:
# slice = fablib.get_slice(name=slice_name)

### Create the Slice

In [4]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Add switch node s1
    s1 = slice.add_node(name=s1_name, site=site_1)
    s1.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s1.set_image(image)
    [s1_iface_h1, s1_iface_h2] = s1.add_component(model='NIC_ConnectX_6', name="s1_switch_nic").get_interfaces()
    s1_iface_h1.set_vlan(vlan=vlan)
    s1_iface_h2.set_vlan(vlan=vlan)


    
    # Add host node h1
    h1 = slice.add_node(name=h1_name, site=site_1)
    h1.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h1.set_image(image)
    [h1_iface,_] = h1.add_component(model='NIC_ConnectX_6', name="h1_nic").get_interfaces()
    
    # Add host node h2
    h2 = slice.add_node(name=h2_name, site=site_2)
    h2.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h2.set_image(image)
    [h2_iface,_] = h2.add_component(model='NIC_ConnectX_5', name="h2_nic").get_interfaces()

    #Add host networks 
    host_net1 = slice.add_l2network(name=net_h1_name, interfaces=[s1_iface_h1, h1_iface])
    host_net2 = slice.add_l2network(name=net_h2_name, interfaces=[s1_iface_h2, h2_iface])
    
    #Submit Slice Request
    slice.submit() #(wait_progress=True)
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    


-----------  ------------------------------------
Slice Name   VCC3_2
Slice ID     a84d0ba0-95ac-46aa-a6b3-0d11a4a4f84d
Slice State  StableOK
Lease End    2022-09-08 04:05:11 +0000
-----------  ------------------------------------

Retry: 11, Time: 139 sec

ID                                    Name    Site    Host                          Cores    RAM    Disk  Image              Management IP    State    Error
------------------------------------  ------  ------  --------------------------  -------  -----  ------  -----------------  ---------------  -------  -------
4f750a76-0c6c-4c3c-bc00-5f2a070e2714  s1      MAX     max-w3.fabric-testbed.net         2      8     100  default_ubuntu_20  63.239.135.70    Active
86dfdd18-46fa-4752-910e-3a1811ad2809  h1      MAX     max-w3.fabric-testbed.net         2      8      10  default_ubuntu_20  63.239.135.116   Active
20f4704b-0586-44d2-a00a-058b5688b4d4  h2      TACC    tacc-w5.fabric-testbed.net        2      8      10  default_ubuntu_20  12

In [10]:
print(host_net1)
print(host_net11)

-----------------  ------
ID
Name               net_h1
Layer
Type
Site
Gateway
L3 Subnet
Reservation State
Error Message
-----------------  ------
-----------------  -------
ID
Name               net_h11
Layer
Type
Site
Gateway
L3 Subnet
Reservation State
Error Message
-----------------  -------


In [6]:
def checkInetConnection(node):
    try:

        #If the node is an IPv6 Node then configure NAT64
        if type(ip_address(node.get_management_ip())) is IPv6Address:
            print("This code only works with ip4, reconfigure nodes")
            return

        #Access non-IPv6 Services
        stdout, stderr = node.execute(f'sudo apt-get update -qq && sudo apt install net-tools && git clone https://github.com/fabric-testbed/jupyter-examples.git')
        # print(stdout)
        # print(stderr)

        stdout, stderr = node.execute(f'ls jupyter-examples')
        print(stdout)
        print(stderr)
        
        stdout, stderr = node.execute(f'ifconfig | grep ens')
        print(stdout)
        print(stderr)


    except Exception as e:
        print(f"Exception: {e}")

In [7]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        checkInetConnection(node)
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

Node:
   Name              : s1
LICENSE
Readme.md
fabric_examples
start_here.ipynb


ens3: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 9000
ens7: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
ens8: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
ens7.1000: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
ens8.1000: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500


Node:
   Name              : h1
LICENSE
Readme.md
fabric_examples
start_here.ipynb


ens3: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 9000
ens7: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500


Node:
   Name              : h2
LICENSE
Readme.md
fabric_examples
start_here.ipynb


ens3: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 9000
ens7: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500




### Print Details

In [8]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print("----")
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        # print(f"   Cores             : {node.get_cores()}")
        # print(f"   RAM               : {node.get_ram()}")
        # print(f"   Disk              : {node.get_disk()}")
        # print(f"   Image             : {node.get_image()}")
        # print(f"   Image Type        : {node.get_image_type()}")
        # print(f"   Host              : {node.get_host()}")
        # print(f"   Site              : {node.get_site()}")
        # print(f"   Management IP     : {node.get_management_ip()}")
        # print(f"   Reservation ID    : {node.get_reservation_id()}")
        # print(f"   Reservation State : {node.get_reservation_state()}")
        ssh = node.get_ssh_command().replace("ssh -i /home/fabric/work/fabric_config/.ssh/id_rsa -J durbek_gafurov_0000000854@bastion-1.fabric-testbed.net","./connect.sh")
        print(f"   SSH Command       : {ssh}")
        # print(f"   Components        :  ")
        # for component in node.get_components():
        #     print(f"      Name             : {component.get_name()}")
        #     print(f"      Details          : {component.get_details()}")
        #     print(f"      Disk (G)         : {component.get_disk()}")
        #     print(f"      Units            : {component.get_unit()}")
        #     print(f"      PCI Address      : {component.get_pci_addr()}")
        #     print(f"      Model            : {component.get_model()}")
        #     print(f"      Type             : {component.get_type()}") 
        print(f"   Interfaces        :  ", end = " ")
        for interface in node.get_interfaces():
        #     print(f"       Name                : {interface.get_name()}")
        #     print(f"           Bandwidth           : {interface.get_bandwidth()}")
            # print(f"           VLAN                : {interface.get_vlan()}")  
            # print(f"           MAC                 : {interface.get_mac()}") 
            print(interface.get_os_interface(), end =" ")
    # for network in slice.get_l2networks():
    #     print("Network:")
    #     print(f"    Name:            {network.get_name()}")
    # print(f"Interface Map: {slice.get_interface_map()}")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

----
Node:
   Name              : s1
   SSH Command       : ./connect.sh ubuntu@63.239.135.70
   Interfaces        :   ens7.1000 ens8.1000 ----
Node:
   Name              : h1
   SSH Command       : ./connect.sh ubuntu@63.239.135.116
   Interfaces        :   ens7 ens8 ----
Node:
   Name              : h2
   SSH Command       : ./connect.sh ubuntu@129.114.110.119
   Interfaces        :   ens8 ens7 

## Configure Nodes


In [11]:
config_threads = {}

In [12]:
host_config_script = "sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git clone https://github.com/p4lang/tutorials.git/" 
def addIP(name,net_name,node_addr,node_subnet):
    node = slice.get_node(name=name)        
    h1_os_iface = node.get_interface(network_name=net_name).ip_addr_add(addr=node_addr, subnet=node_subnet)
    config_thread = node.execute_thread(host_config_script)
    config_threads[node]=config_thread
    return node
    
try:
    h1_subnet=IPv4Network('10.0.1.0/24')
    h1 = addIP(h1_name,net_h1_name,h1_ip,h1_subnet)
    h2 = addIP(h2_name,net_h2_name,h2_ip,h1_subnet)


except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [13]:
def checkIP(node):
    try:        
        stdout, stderr = node.execute(f'ifconfig | grep "inet 10.0."')
        print(stdout)
        print(stderr)
    except Exception as e:
        print(f"Exception: {e}")

In [14]:
try: 
    for node, thread in config_threads.items():
        stdout, stderr = thread.result()
        print(f"Config thread node {node.get_name()} complete")
        checkIP(node)
        # print(f"stdout: {stdout}")
        # print(f"stderr: {stderr}")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 
config_threads = {}

Config thread node h1 complete
        inet 10.0.1.1  netmask 255.255.255.0  broadcast 0.0.0.0


Config thread node h2 complete
        inet 10.0.1.2  netmask 255.255.255.0  broadcast 0.0.0.0




## Configure Switches

Use ssh to configure the ifaces on the switches. This step requires testing the interfaces to figure out which interface is connected to which network.


#### Setup P4 Docker



Below are commands to let sudo work with the global proxy.

In [15]:
h1Mac = h1.get_interface(network_name=net_h1_name).get_mac()
h2Mac = h2.get_interface(network_name=net_h2_name).get_mac()


print(h1Mac,h2Mac)
def printToFile(text,filename):
    text_file = open(filename, "w")
    n = text_file.write(text)
    text_file.close()
s1_command = """table_clear MyEgress.swtrace
table_clear MyIngress.ipv4_lpm 
table_set_default MyEgress.swtrace add_swtrace 1
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.1.1/32 => {} 1
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.1.2/32 => {} 2
""".format(h1Mac, h2Mac)

printToFile(s1_command, "./scripts/s1__command.txt")
print(s1_command, "./scripts/s1command.txt")

04:3F:72:FE:A6:70 B8:CE:F6:18:F8:FA
table_clear MyEgress.swtrace
table_clear MyIngress.ipv4_lpm 
table_set_default MyEgress.swtrace add_swtrace 1
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.1.1/32 => 04:3F:72:FE:A6:70 1
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.1.2/32 => B8:CE:F6:18:F8:FA 2
 ./scripts/s1command.txt


In [16]:
def configureSwitch(name,network_names):
    try:
        node = slice.get_node(name=name)
        os_ifaces = ""
        for net_name in network_names:
            iface = node.get_interface(network_name=net_name).get_os_interface()
            if iface is None:
                print("iface is none")
                return
            os_ifaces=os_ifaces+iface+" "

        file_attributes = node.upload_file('./scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
        command=f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh {os_ifaces}  > /tmp/script.log 2>&1' && tail /tmp/script.log"
        config_threads[node] = node.execute_thread(command)
        print(command)


    except Exception as e:
        print(f"Error: {e}")
        traceback.print_exc()

In [17]:
configureSwitch(s1_name,[net_h1_name,net_h2_name]) 


chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh ens7.1000 ens8.1000   > /tmp/script.log 2>&1' && tail /tmp/script.log


In [18]:
try: 
    for node, thread in config_threads.items():
        stdout, stderr = thread.result()
        print(f"Config thread node {node.get_name()} complete")
        print(f"stdout: {stdout}")
        print(f"stderr: {stderr}")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 


Config thread node s1 complete
stdout: V1Switch(
MyParser(),
MyVerifyChecksum(),
MyIngress(),
MyEgress(),
MyComputeChecksum(),
MyDeparser()
) main;
simple_switch --interface 1@ens7.1000 --interface 2@ens8.1000 ~/P4-VCC/exercises/mri_port_forward/mri.json
done!

stderr: 


### Confgure P4 Switch Tables

Edit sX_commands.txt to change the values

In [19]:

for switch_name in [s1_name]:
    switch_node = slice.get_node(name=switch_name)
    management_ip_switch = str(switch_node.get_management_ip())
    print("Swtitch Name        : {}".format(switch_node.get_name()))
    print("Management IP    : {}".format(management_ip_switch))
    
    #Configure P4 Tables
    cmd_file=f'{switch_name}__command.txt'
    print(cmd_file)
    file_attributes = switch_node.upload_file(f'scripts/{cmd_file}',cmd_file)
    print("file_attributes: {}".format(file_attributes))

    stdout = switch_node.execute(f"sudo sh -c 'cat {cmd_file} | docker exec -i fabric_p4 simple_switch_CLI'")
    print("stdout: {}".format(stdout))

Swtitch Name        : s1
Management IP    : 63.239.135.70
s1__command.txt
file_attributes: -rw-rw-r--   1 1000     1000          264 07 Sep 04:21 ?
stdout: ('Obtaining JSON from switch...\nDone\nControl utility for runtime P4 table manipulation\nRuntimeCmd: RuntimeCmd: RuntimeCmd: Setting default action of MyEgress.swtrace\naction:              add_swtrace\nruntime data:        00:00:00:01\nRuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm\nmatch key:           LPM-0a:00:01:01/32\naction:              ipv4_forward\nruntime data:        04:3f:72:fe:a6:70\t00:01\nEntry has been added with handle 0\nRuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm\nmatch key:           LPM-0a:00:01:02/32\naction:              ipv4_forward\nruntime data:        b8:ce:f6:18:f8:fa\t00:02\nEntry has been added with handle 1\nRuntimeCmd: \n', '')


## The switches are now configured and running. Now we are going to send packets over the switches.

We are going to use `send.py` and `receive.py`. We are going to re-upload them to the servers and use them. Make sure to modify the interface names in the script accordingly.

In [20]:
h1.upload_file('scripts/send.py', 'tutorials/exercises/basic_tunnel/send_modified.py')
h1.upload_file('scripts/receive.py', 'tutorials/exercises/basic_tunnel/receive_modified.py')
h2.upload_file('scripts/send.py', 'tutorials/exercises/basic_tunnel/send_modified.py')
h2.upload_file('scripts/receive.py', 'tutorials/exercises/basic_tunnel/receive_modified.py')

<SFTPAttributes: [ size=1498 uid=1000 gid=1000 mode=0o100664 atime=1662524514 mtime=1662524514 ]>

## H1 sends packet, H2 recieves

In [25]:
h1.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py 10.0.1.2 "message100" 50\'')
print(h2.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')[0])

sniffing on ens7



## H1 sends packet, H11 recieves

In [26]:
h1.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py 10.0.1.11 "message100" 50\'')
print(h11.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')[0])

sniffing on ens7



## H2 sends packet, H1 recieves

In [27]:
h2.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py 10.0.1.1 "message100" 50\'')
print(h1.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')[0])

sniffing on ens7



## H22 sends packet, H11 recieves

In [28]:

h22.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py 10.0.1.11 "message100" 50\'')
print(h11.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')[0])

sniffing on ens7



In [30]:

h1.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py 10.0.1.11 "message100" 50\'')
print(h11.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')[0])

sniffing on ens7



## Delete Slice

In [ ]:
# try:
#     slice = fablib.get_slice(name=slice_name)
#     slice.delete()
# except Exception as e:
#     print(f"Fail: {e}")
#     traceback.print_exc()